In [101]:
import oggm 
from oggm import cfg, workflow, utils
from oggm.workflow import execute_entity_task
import geopandas as gpd
import salem, os, netCDF4, logging
import numpy as np
import pandas as pd
import xarray as xr
import datetime
from datetime import datetime
log = logging.getLogger(__name__)

In [2]:
cfg.initialize()
cfg.PARAMS['continue_on_error'] = True
cfg.PATHS['climate_file'] = "/home/david/Desktop/CCSM4_midHolocene_data/127ka.nc"
cfg.PARAMS['use_multiprocessing'] = True
PI_path = "/home/david/Desktop/CCSM4_midHolocene_data/pi.nc"
rgi_version = '5'
rgi_region = '17'
cfg.PATHS['working_dir'] = "/home/david/OGGM_WORKING_DIRECTORY/"
path = utils.get_rgi_intersects_region_file(rgi_region, version=rgi_version)
cfg.set_intersects_db(path)

2018-09-12 15:59:48: oggm.cfg: Parameter file: /home/david/oggm/oggm/params.cfg


In [3]:
entity = gpd.GeoDataFrame.from_file('/home/david/OGGM/rgi/RGIV5/06_rgi50_Iceland/06_rgi50_Iceland.shp')
gdirs = oggm.workflow.init_glacier_regions(entity)

2018-09-12 15:59:51: oggm.workflow: Multiprocessing: using all available processors (N=12)


In [4]:
gdir = gdirs[0]
gdir

<oggm.GlacierDirectory>
  RGI id: RGI50-06.00001
  Region: 06: Iceland
  Subregion: 06-00: Iceland
  Glacier type: Glacier
  Terminus type: Land-terminating
  Area: 4.903 km2
  Lon, Lat: (-23.7852, 64.8174)
  Grid (nx, ny): (118, 117)
  Grid (dx, dy): (41.0, -41.0)

In [142]:
fpath = gdir.get_filepath('cesm_data')
with netCDF4.Dataset(fpath, mode='r') as nc:
    time = nc.variables['time']
    time = netCDF4.num2date(time[:], time.units)
time

array([datetime.datetime(1950, 10, 1, 0, 0),
       datetime.datetime(1950, 11, 1, 0, 0),
       datetime.datetime(1950, 12, 1, 0, 0),
       datetime.datetime(1951, 1, 1, 0, 0),
       datetime.datetime(1951, 2, 1, 0, 0),
       datetime.datetime(1951, 3, 1, 0, 0),
       datetime.datetime(1951, 4, 1, 0, 0),
       datetime.datetime(1951, 5, 1, 0, 0),
       datetime.datetime(1951, 6, 1, 0, 0),
       datetime.datetime(1951, 7, 1, 0, 0),
       datetime.datetime(1951, 8, 1, 0, 0),
       datetime.datetime(1951, 9, 1, 0, 0)], dtype=object)

In [143]:
fpath = gdir.get_filepath('climate_monthly')
with netCDF4.Dataset(fpath, mode='r') as nc:
    time = nc.variables['time']
    time = netCDF4.num2date(time[:], time.units)
time

array([datetime.datetime(1901, 10, 1, 0, 0),
       datetime.datetime(1901, 11, 1, 0, 0),
       datetime.datetime(1901, 12, 1, 0, 0), ...,
       datetime.datetime(2015, 7, 1, 0, 0),
       datetime.datetime(2015, 8, 1, 0, 0),
       datetime.datetime(2015, 9, 1, 0, 0)], dtype=object)

In [131]:
#Put this in the function def (see process_cesm_data)
filesuffix=''

if not (('climate_file' in cfg.PATHS) and
        os.path.exists(cfg.PATHS['climate_file'])):
    raise IOError('Custom climate file not found')

#open dataset for precp use
fpath = cfg.PATHS['climate_file']
xr_ccsm = xr.open_dataset(fpath, decode_times=False)
#open dataset for tmp use
xr_ccsm_ts = xr.open_dataset(fpath)
#repeating for pi
xr_pi = xr.open_dataset(PI_path, decode_times=False)

# selecting location
lon = gdir.cenlon
lat = gdir.cenlat

#Setting the longitude to a 0-360 grid [I think...] "CESM files are in 0-360"
if lon <= 0:
    lon += 360

#"take the closest"
#"TODO: consider GCM interpolation?"
precp = xr_ccsm.PRECC.sel(lat=lat, lon=lon, method='nearest') + xr_ccsm.PRECL.sel(lat=lat, lon=lon, method='nearest')
temp = xr_ccsm_ts.TS.sel(lat=lat, lon=lon, method='nearest')
precp_pi = xr_pi.PRECC.sel(lat=lat, lon=lon, method='nearest') + xr_pi.PRECL.sel(lat=lat, lon=lon, method='nearest')
temp_pi = xr_pi.TS.sel(lat=lat, lon=lon, method='nearest')

#convert temp from K to C
temp = temp - 273.15
temp_pi = temp_pi - 273.15

#Take anomaly for CCSM data (with preindustrial control)
for i in range(12):
    temp.values[i] = temp.values[i] - temp_pi.values[i]
for i in range(12):
    precp.values[i] = precp.values[i] - precp_pi.values[i]

In [132]:
#from normal years to hydrological years
sm = cfg.PARAMS['hydro_month_'+gdir.hemisphere]
em = sm - 1 if (sm > 1) else 12
y0 = int(pd.to_datetime(str(temp.time.values[0])).strftime('%Y'))
y1 = int(pd.to_datetime(str(temp.time.values[-1])).strftime('%Y'))
#time string for temp/precip (hydro years)
time = pd.period_range('{}-{:02d}'.format(y0, sm),'{}-{:02d}'.format(y1, em), freq='M')

### At this point, the ccsm3 input year still needs to be reformatted into the proper order. 
This is where I've struggled the most, espeically making sure it reformats to the correct hydroyear for the hemisphere. 

The `time` string represents the proper time order/format, and will be the correct hydroyear b/c of the `cfg.PARAMS['hydro_month_'+gdir.hemisphere]

In [133]:
#reorder single year of equil data & add correct time
#calculate place in array to concat from (2 for ccsm data)
conc_start = sm-2
conc_end = sm-14

temp_hydro = xr.concat([temp[conc_start:],temp[:conc_end]],dim="time")
precp_hydro = xr.concat([precp[conc_start:],precp[:conc_end]],dim="time")
temp_hydro['time'] = time
precp_hydro['time'] = time

In [134]:
temp = temp_hydro
precp = precp_hydro

# Workaround for https://github.com/pydata/xarray/issues/1565
temp['month'] = ('time', time.month)
precp['month'] = ('time', time.month)
temp['year'] = ('time', time.year)
temp['year'] = ('time', time.year)
ny, r = divmod(len(temp.time), 12)
assert r == 0

In [135]:
#Convert m s-1 to mm mth-1 (for precp)
ndays = np.tile(cfg.DAYS_IN_MONTH, y1-y0)
precp = precp * ndays * (60 * 60 * 24 * 1000)

In [136]:
#"Get CRU to apply the anomaly to"
fpath = gdir.get_filepath('climate_monthly')
ds_cru = xr.open_dataset(fpath)

#"Add the climate anomaly to CRU clim"
dscru = ds_cru.sel(time=slice('1961', '1990'))

In [137]:
# temp
loc_temp = dscru.temp.groupby('time.month').mean()
#Oct-Sept format preserved
ts_tmp = temp.groupby(temp.month) + loc_temp

In [138]:
#for prcp
loc_pre = dscru.prcp.groupby('time.month').mean()
ts_pre = precp.groupby(precp.month) + loc_pre

In [139]:
#load dates into save format
fpath = cfg.PATHS['climate_file']
dsindex = salem.GeoNetcdf(fpath, monthbegin=True)
time1 = dsindex.variables['time']

In [140]:
#weird recursive way to getting the dates in the correct format to save 
#only nessisary for 1 year of data, in order to rearrange the months and 
#get the correct year.
time_array = [datetime(temp.time.year[i], temp.time.month[i],1) for i in range(12)]
time_nums = netCDF4.date2num(time_array, time1.units, calendar='noleap')
time2 = netCDF4.num2date(time_nums[:], time1.units, calendar='noleap')

In [141]:
assert np.all(np.isfinite(ts_pre.values))
assert np.all(np.isfinite(ts_tmp.values))

#"back to -180 - 180"
loc_lon = precp.lon if precp.lon <= 180 else precp.lon - 360

#write to netcdf
gdir.write_monthly_climate_file(time2, ts_pre.values, ts_tmp.values, 
                                float(dscru.ref_hgt), loc_lon, 
                                precp.lat.values, 
                                time_unit=time1.units, 
                                file_name='cesm_data', 
                                filesuffix=filesuffix)

#dsindex._nc.close()
xr_ccsm.close()
xr_ccsm_ts.close()
xr_pi.close()
ds_cru.close()

#logger message
log.info("%s", gdir.rgi_id)

2018-09-12 17:27:01: __main__: RGI50-06.00001
